# Exercise 02 - CDS Feature Comparisons <img src="images/JHI_STRAP_Web.png" style="width: 150px; float: right;">

## Introduction

We often have particular genes/proteins of interest, and want to identify these - or equivalent sequences - in our newly-sequenced genomes. We may also want to identify corresponding sequences in bulk across our newly-sequenced genomes. In this notebook, we will look at three methods (among many others!) of identifying equivalent sequence features in genomes, in bulk.

The methods we will look at can broadly be considered to be of three different types:

* one-way pairwise comparison
* two-way pairwise comparison
* clustering

and they will give different results, for the same data.

## Running cells in this notebook

This is an interactive notebook, which means you are able to run the code that is written in each of the cells.

To run the code in a cell, you should:

1. Place your mouse cursor in the cell, and click (this gives the cell *focus*) to make it active
2. Hold down the `Shift` key, and press the `Return` key.

If this is successful, you should see the input marker to the left of the cell change from

```
In [ ]:
```

to (for example)

```
In [1]:
```

and you may see output appear below the cell.

### Requirements

<div class="alert alert-success">
To complete this exercise, you will need:
<ul>
<li>an active internet connection
<li>a local installation of [`BLAST+`](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastDocs&DOC_TYPE=Download)
<li>a local installation of [`OrthoFinder`])(https://github.com/davidemms/OrthoFinder/releases)
</ul>
</div>

### Related online documentation/publications/software

**Software**
* [CRB-BLAST](https://github.com/cboursnell/crb-blast) - conditional reciprocal best BLAST

**Publications**
* Aubrey *et al.* (2014) *PLoS Genet.* [doi:10.1371/journal.pgen.1004365](http://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1004365)

**Blogs**
* [On Reciprocal Best Blast Hits](http://armchairbiology.blogspot.co.uk/2012/07/on-reciprocal-best-blast-hits.html)

## One-Way Best `BLAST` matches (BBH)

It is still common to see one-way matches used - even if only informally, or as a first attempt - as a means of identifying equivalent proteins/features in a genome. In this section, we'll carry out a one-way `BLAST` search between the protein complements of *P. syringae* B728a and *P. fluorescens* NCIMB 11764, and inspect the results graphically.

### Performing the `BLASTP` query

We will use the `blastp` command at the terminal to use every protein sequence in the *P. syringae* B728a annotation as a query against the predicted proteome of *P. fluorescens* NCIMB 11764. We will use some custom settings to make our analysis easier to carry out.

<div class="alert alert-warning">
<ul>
<li> We will want to limit our matches to only the best hit, so we specify `-max_target_seqs 1` 
<li> We want our output in tab-separated tabular particular format so we can import it easily, and we want some specific non-standard columns (e.g. query sequence coverage) in that table so we can carry out some useful calculations and visualisation. We therefore specify `-outfmt "6 qseqid sseqid qlen slen length nident pident qcovs evalue bitscore"`
<li> To make the comparisons quicker, we should create `BLAST` databases for each of the three proteomes, with the `makeblastdb` command. 
</ul>
</div>

The relevant `BLAST` databases have already been created for you to save time (using the `scripts/02-cds_feature_comparisons.sh` script), and the results are in the `pseudomonas_blastp` directory:

```
$ tree ./pseudomonas_blastp
./pseudomonas_blastp
├── GCF_000012245.1_ASM1224v1_protein.phr
├── GCF_000012245.1_ASM1224v1_protein.pin
├── GCF_000012245.1_ASM1224v1_protein.psq
├── GCF_000293885.2_ASM29388v3_protein.phr
├── GCF_000293885.2_ASM29388v3_protein.pin
├── GCF_000293885.2_ASM29388v3_protein.psq
├── GCF_000988485.1_ASM98848v1_protein.phr
├── GCF_000988485.1_ASM98848v1_protein.pin
└── GCF_000988485.1_ASM98848v1_protein.psq
```

To carry out the one-way `BLASTP` search of *P. syringae* B728a against *P. fluorescens* NCIMB 11764, execute the following command in the terminal:

```
blastp -query pseudomonas/GCF_000988485.1_ASM98848v1_protein.faa \
       -db pseudomonas_blastp/GCF_000293885.2_ASM29388v3_protein \
       -max_target_seqs 1 \
       -outfmt "6 qseqid sseqid qlen slen length nident pident qcovs evalue bitscore" \
       -out pseudomonas_blastp/B728a_vs_NCIMB_11764.tab
```

This will take a few minutes to complete, so to save time the comparison has already been made for you, with the result file being placed in `pseudomonas_blastp/B728a_vs_NCIMB_11764.tab`.

### Importing and visualising the results

The `Python` module `helpers` is included in this directory, to provide useful helper functions so that we can read and view the `BLASTP` output generated above. To make the functions available, we import it by running the `Python` code cell below.

<div class="alert alert-warning">
<b>NOTE:</b> The `%pylab inline` "magic" below allows us to see plots of the `BLAST` data we load.
</div>

In [ ]:
%pylab inline
# Import helper module
from helpers import ex02

The first thing we do is load in the `BLASTP` output we generated, so that we can plot some of the key features. We do that using the `ex02.read_data()` function in the cell below:

In [ ]:
# Load one-way BLAST results into a data frame called data_fwd
data_fwd = ex02.read_data("pseudomonas_blastp/B728a_vs_NCIMB_11764.tab")

<div class="alert alert-warning">
<b>NOTE:</b> In the cell below, the `data.head()` function shows us the first few lines of the one-way `BLASTP` results, one per match; the `data.describe()` function shows us some summary data for the table.
</div>

In [ ]:
# Show first few lines of the loaded data
data_fwd.head()

In [ ]:
# Show descriptive statistics for the table
data_fwd.describe()

There are 5265 rows in this table, one for each of the query protein sequences in the *P. syringae* B728a annotation.

We can look at the distribution of values in the dataframe rows using the `.hist()` method for any column of interest. For example, `data_fwd.subject_length.hist()` plots a histogram of the values in the `subject_length` column.

<div class="alert alert-warning">
<b>NOTE:</b> The `bins=100` option sets the number of value bins used in the histogram
</div>

In [ ]:
# Plot a histogram of alignment lengths for the BLAST data
data_fwd.alignment_length.hist(bins=100)

In [ ]:
# Plot a histogram of percentage identity for the BLAST data
data_fwd.identity.hist(bins=100)

In [ ]:
# Plot a histogram of query_coverage for the BLAST data
data_fwd.query_coverage.hist(bins=100)

In [ ]:
# Plot a histogram of percentage coverage for the BLAST data
data_fwd.subject_coverage.hist(bins=100)

From these histograms, we can see that:

* most alignments are shorter than 500 amino acids long
* that the alignment covers more than 80% of nearly all queries
* that the alignment covers more than 80% of nearly all best matches to each query
* there is a bimodal distribution of sequence identity: one where best matches peak at ≈80% identity, and one where best matches peak at ≈30% identity

<div class="alert alert-warning">
<ul>
<li><b>What size are most one-way best `BLAST` alignments?</b>
<li><b>What is the typical query coverage?</b>
<li><b>What is the typical subject coverage?</b>
<li><b>What is the typical best `BLAST` match identity?</b>
</ul>
</div>

We can view the relationship between query coverage and subject coverage, and query coverage and match identity for these one-way best `BLAST` hits by plotting a 2D histogram, with the helper function `ex02.plot_hist2d()` in the cell below.

In [ ]:
# Plot 2D histogram of subject sequence (match) coverage against query
# sequence coverag
ex02.plot_hist2d(data_fwd.query_coverage, data_fwd.subject_coverage,
                "one-way query COV", "one-way subject COV", 
                "one-way coverage comparison")
ex02.plot_hist2d(data_fwd.query_coverage, data_fwd.identity,
                "one-way query COV", "one-way match PID", 
                "one-way coverage/identity comparison")

<div class="alert alert-warning">
<ul>
<li>**What is the query/subject coverage for most one-way best `BLAST` matches?**
<li>**Why do some one-way `BLAST` matches not have the same coverage for query and subject?**
<li>**What is the typical query coverage of a high percentage identity match?**
<li>**What is the typical query coverage of a low percentage identity match?**
</ul>
</div>

<div class="alert alert-danger" role="alert">
<b>QUESTION:</b><br />
<b>Do one-way best `BLAST` matches always identify equivalent proteins?</b>
</div>

## Reciprocal (Two-Way) Best `BLAST` matches (RBBH)

To perform a reciprocal BLAST search between two sets of proteins `S1` and `S2` (say), we need to carry out the *forward* search of `S1` vs `S2`, and the *reverse* search `S2` vs `S1`.

Reciprocal best `BLAST` matches are those where the sequence `G(S1)` (a gene/CDS from sequence set `S1`) used as a query makes its best `BLAST` match to sequence `G(S2)` (a gene/CDS from sequence set `S2`), and when sequence `G(S2)` is used as a query it makes its best match to sequence `G(S1)` (see figure below).

![Schematic of RBBH](images/rbbh.png)

We carried out the forward search above, for *P. syringae* B728a (our sequence set `S1`) against *P. fluorescens* NCIMB 11764 (our sequence set `S2`), and now we will carry out the corresponding reverse search by executing the command below at the terminal:

```
blastp -query pseudomonas/GCF_000293885.2_ASM29388v3_protein.faa \
       -db pseudomonas_blastp/GCF_000988485.1_ASM98848v1_protein \
       -max_target_seqs 1 \
       -outfmt "6 qseqid sseqid qlen slen length nident pident qcovs evalue bitscore" \
       -out pseudomonas_blastp/NCIMB_11764_vs_B728a.tab
```

As before, this would few minutes to complete, so to save some time the comparison has already been made for you, with the result file being placed in `pseudomonas_blastp/NCIMB_11764_vs_B728a.tab`.

We'll load the results into the variable `data_rev` using the helper function `ex02.read_data()` in the cell below.

In [ ]:
# Load one-way BLAST results into a data frame called data_fwd
data_rev = ex02.read_data("pseudomonas_blastp/NCIMB_11764_vs_B728a.tab")

<div class="alert alert-warning">
<b>NOTE:</b> You could inspect `data_rev` using the `.head()` and `.describe()` methods, just as you did for `data_fwd`
</div>

The `ex02` module provides a function called `find_rbbh()` which calculates reciprocal best BLAST hits from forward and reverse `BLAST` searches. The calculation can be performed by executing the cell below.

In [ ]:
# Calculate RBBH for the two Pseudomonas datasets
# This returns three dataframes: df1 and df2 are the forward and reverse BLAST
# results (filtered, if any filters were used), and rbbh is the dataframe of
# reciprocal best BLAST hits
df1, df2, rbbh = ex02.find_rbbh(data_fwd, data_rev)

We can inspect the dataframe of RBBH using the `.head()` and `.describe()` methods, by executing the cells below.

In [ ]:
# Peek at the first few lines of the RBBH results
rbbh.head()

In [ ]:
# Show summary statistics for RBBH
rbbh.describe()

It is inevitable that the RBBH set will have the same or fewer protein pairs in it, than the number of proteins in the smallest of the forward and reverse protein sets. But how many proteins have been filtered in this comparison? We can find out by executing the cell below.

In [ ]:
# Report the size of each of the forward and reverse input, and rbbh output dataframes
s = '\n'.join(["Forward BLAST input: {0} proteins",
               "Reverse BLAST input: {1} proteins",
               "RBBH output: {2} proteins"])
print(s.format(len(data_fwd), len(data_rev), len(rbbh)))
print("(min difference = {0})".format(min(len(data_fwd), len(data_rev)) - len(rbbh)))

<div class="alert alert-warning">
Approximately what proportion of best `BLAST` matches have been discarded?
</div>

### Visualising RBBH output
We can get a better idea of what this processing has done by looking at a visual representation of the percentage identity and coverage of RBBH, compared to the (forward) one-way matches. We can do this by executing the cells below.

First, let's look at the percentage identity of best `BLAST` matches:

In [ ]:
# Histogram of forward match percentage identity (one-way)
data_fwd.identity.hist(bins=100)

In [ ]:
# Histogram of forward match percentage identity (RBBH)
rbbh.identity_x.hist(bins=100)

<div class="alert alert-warning">
<b>What has been the effect of excluding best matches that do not have an RBBH reverse match?</b>
</div>

Next, we can inspect the query and subject coverage of RBBH results, compared to the one-way forward `BLAST` matches by executing the cell below.

In [ ]:
# Plot 2D histograms of query coverage against subject coverage for the 
# one-way forward matches, and those retained after calculating RBBH
ex02.plot_hist2d(data_fwd.query_coverage, data_fwd.subject_coverage,
                "one-way query COV", "one-way subject COV", 
                "one-way coverage comparison")
ex02.plot_hist2d(rbbh.query_coverage_x, rbbh.subject_coverage_x,
                "RBBH (fwd) query COV", "RBBH (fwd) subject COV", 
                "RBBH coverage comparison")

<div class="alert alert-warning">
<ul>
<li><b>Which one-way matches have been excluded by carrying out RBBH?</b><br />
<li><b>What is the biological significance of excluding those matches?</b>
<li><b>What would be a reasonable filter to exclude the remaining suspect matches?</b>
</ul>
</div>

### Filtering RBBH output
The `find_rbbh()` function allows us to apply cutoff filters on percentage identity or coverage (or both) for an RBBH match - this, and visualisation of the results is done in the cells below.

<div class="alert alert-warning">
<b>NOTE:</b> There is a software tool ([`CRB-BLAST`](https://github.com/cboursnell/crb-blast) - Conditional Reciprocal Best BLAST) available that statistically evaluates an E-value cutoff for BLAST matches, in order to improve accuracy of ortholog assignment.
</div>

In [ ]:
# Calculate ID and coverage-filtered RBBH for the two Pseudomonas datasets
# This returns three dataframes: df1_filtered and df2_filtered are the 
# filtered forward and reverse BLAST results , and rbbh_filtered is the
# dataframe of reciprocal best BLAST hits
df1_filtered, df2_filtered, rbbh_filtered = ex02.find_rbbh(data_fwd, data_rev, pid=40, cov=70)

# Histogram of forward match percentage identity (RBBH, filtered)
rbbh_filtered.identity_x.hist(bins=100)

In [ ]:
# Plot 2D histograms of query coverage against subject coverage for the 
# one-way forward matches retained after calculating RBBH and
# filtering on percentage identity and coverage
ex02.plot_hist2d(rbbh_filtered.query_coverage_x, rbbh_filtered.subject_coverage_x,
                "filtered RBBH (fwd) query COV", "filtered_RBBH (fwd) subject COV", 
                "filtered RBBH coverage comparison")

### Visualising RBBH with `ACT`

Finally for this exercise, we will visualise the RBBH between *P. syringae* B728a and *P. fluorescens* NCIMB 11764 using `ACT` (as in exercise 01), comparing the output to that obtained by a `BLASTN` comparison of the chromosomes.

First, we need to generate an output file describing our (filtered) RBBH that `ACT` can read. We do this by executing the cell below. This does two things:

* Gets the locations of protein features on the chromosome of each organism from a `.gbff` file, using the helper function `read_genbank()`, putting them in a variable called `features`.
* Writes the RBBH to a `.crunch` format file (`pseudomonas_blastp/B728a_rbbh_NCIMB_11764.crunch`), which `ACT` can read.


In [ ]:
# Read feature locations for each Pseudomonas file
features = ex02.read_genbank("pseudomonas/GCF_000988485.1_ASM98848v1_genomic.gbff",
                             "pseudomonas/GCF_000293885.2_ASM29388v3_genomic.gbff")

# Write a .crunch file of filtered RBBH for the Pseudomonas comparisons
ex02.write_crunch(rbbh_filtered, features,
                  fwd="GCF_000988485.1_ASM98848v1_genomic",
                  rev="GCF_000293885.2_ASM29388v3_genomic",
                  outdir="pseudomonas_blastp",
                  filename="B728a_rbbh_NCIMB_11764.crunch")

Now we can load the two genome sequences, the `BLASTN` comparison, and the RBBH comparisons into `ACT`.

If `ACT` is not already running, we start it at the terminal as before:

```
act &
```

and open the file dialog, expanding it to show five fields, as in the previous exercise. First, select `File -> Open` to bring up the file selection dialog box.

![Empty ACT file selection dialog, 3 fields](images/act1.png)

Then click on the `more files...` button once, to give us two more fields.

![Empty ACT file selection dialog, 5 fields](images/act2.png)

Now we add the sequence files, placing the *P. syringae* B728a genome sequence (`pseudomonas/GCF_000988485.1_ASM98848v1_genomic.fna`) in `Sequence file 2`, and the *P. fluorescens* NCIMB 11764 genome (`pseudomonas/GCF_000293885.2_ASM29388v3_genomic.fna`) in two slots: `Sequence file 1` and `Sequence file 3`, as shown.

![](images/act6.png)

Next, we add the RBBH output file `pseudomonas_blastp/B728a_rbbh_NCIMB_11764.crunch` to the field `Comparison file 1`:

![](images/act7.png)

And finally, we use the `BLASTN` comparison file (`pseudomonas_blastn/B728a_vs_NCIMB_11764.tab`) we prepared in the earlier exercise, adding it to the field `Comparison file 2`:

![](images/act8.png)

and then click on the `Apply` button. This produces the initial alignment:

![](images/act9.png)

which we can zoom in on to inspect differences between the `BLASTN` and `BLASTP` comparisons.

![](images/act10.png)

<div class="alert alert-warning">
<ul>
<li><b>How do the `BLASTN` and RBBH alignments compare to each other, in general?</b><br />
<li><b>What kinds of differences are there?</b>
<li><b>What sorts of biologically useful information can you get from each alignment that is not available from the other alignment?</b>
</ul>
</div>